# LoRA

## Step 1: Import resources

Import libraries and connect to HuggingFace


In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
write_token = userdata.get('WRITE_TOKEN')
finegrained_token = userdata.get('FINEGRAINED_TOKEN')

In [ ]:
import huggingface_hub
huggingface_hub.login(token=hf_token)

In [ ]:
!pip install huggingface-cli
!huggingface-cli login

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install datasets evaluate
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -U bitsandbytes
from transformers import BitsAndBytesConfig
import math
import torch
import pathlib
from pathlib import Path
from typing import Union, Dict, List, Tuple, Optional
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import MultiheadAttention
from torch.nn import functional as F
from torch import Tensor
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR

from transformers import RobertaTokenizer, RobertaTokenizerFast, RobertaModel, pipeline, AutoTokenizer, AutoModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelForMaskedLM, AutoTokenizer, AutoModelForSequenceClassification
from transformers.models.roberta.modeling_roberta import RobertaSelfAttention

from datasets import load_dataset, Dataset

from peft import LoraConfig, LoraModel, PeftModel, get_peft_model, TaskType

import evaluate
import tabulate
import tokenizers
import json
import random
import os

from torch.utils.data import DataLoader
from tqdm import tqdm

model_checkpoint = "roberta-base"

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vpoe_67q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vpoe_67q
  Resolved https://github.com/huggingface/transformers to commit 5615a393691c81e00251e420c73e4d04c6fe22e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10214358 sha256=10c5b8fe8b45ec68f6fd240f340b85ec873226fa1e925d1624a5937a14b812a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-imx8jlvl/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attemptin

In [ ]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 2: Configure LoRA
Use the `peft` library to configure LoRA.

In [ ]:
winogender_path = "drive/MyDrive/DATASCI 266 Final Project/winogender-schema/data/templates.tsv"
data = pd.read_csv(winogender_path, sep="\t")
print(data.head())

  occupation(0) other-participant(1)  answer  \
0    technician             customer       1   
1    technician             customer       0   
2    accountant             taxpayer       1   
3    accountant             taxpayer       0   
4    supervisor             employee       1   

                                            sentence  
0  The $OCCUPATION told the $PARTICIPANT that $NO...  
1  The $OCCUPATION told the $PARTICIPANT that $NO...  
2  The $PARTICIPANT met with the $OCCUPATION to g...  
3  The $PARTICIPANT met with the $OCCUPATION to f...  
4  The $OCCUPATION gave the $PARTICIPANT feedback...  


In [ ]:
# Preprocess the data
def preprocess_function(batch):
  sentences = []
  gender_labels = []
  for sentence, occupation, participant, answer in zip(
      batch['sentence'],
      batch['occupation(0)'],
      batch['other-participant(1)'],
      batch['answer']):
    new_sentence = sentence.replace("$OCCUPATION", occupation)
    new_sentence = new_sentence.replace("$PARTICIPANT", participant)
    pronoun = "he" if answer == 1 else "she"
    new_sentence = new_sentence.replace("$NOM_PRONOUN", pronoun)
    gender_labels.append(0 if pronoun == "he" else 1)
    sentences.append(new_sentence)

  return {"text": sentences, "gender_labels": gender_labels}

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(data)
dataset = dataset.map(preprocess_function, batched = True, remove_columns = ["occupation(0)", "other-participant(1)", "answer", "sentence"])
print(dataset[0])

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

{'text': 'The technician told the customer that he could pay with cash.', 'gender_labels': 0}


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize the dataset
def tokenize_function(batch):
  encoding = tokenizer(batch['text'], truncation=True, padding="max_length", max_length=128)
  encoding['labels'] = encoding['input_ids'].copy()
  encoding['gender_labels'] = batch['gender_labels']
  return encoding

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print(tokenized_dataset)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Dataset({
    features: ['gender_labels', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 120
})


In [ ]:
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm

def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 884,736 || all params: 125,582,169 || trainable%: 0.7045


In [ ]:
training_args = TrainingArguments(
    output_dir="./lora-winogender",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=5,
    save_steps=500,
    eval_strategy="no",
    #gradient_accumulation_steps=4
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True
)

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size = training_args.per_device_train_batch_size,
    collate_fn = data_collator,
    shuffle = True
)

val_dataloader = DataLoader(
    tokenized_dataset,
    batch_size = training_args.per_device_train_batch_size,
    collate_fn = data_collator,
    shuffle = False
)

In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    model.to(device)
    total_loss = 0
    num_batches = 0
    prob_diffs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            if torch.isnan(batch['input_ids']).any() or torch.isnan(batch['attention_mask']).any():
                print("NaN detected in input batch.")
                continue
            gender_labels = batch['gender_labels'].to(device)
            batch = {k: v.to(device) for k, v in batch.items() if k != 'gender_labels'}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            if torch.isnan(loss):
                print("NaN detected in loss. Skipping batch.")
                continue
            total_loss += loss.item()
            num_batches += 1

            # gender_labels = batch['gender_labels']
            group1_mask = gender_labels == 0
            group2_mask = gender_labels == 1

            prob_group1 = torch.softmax(logits[group1_mask], dim=-1).mean().item() if group1_mask.sum() > 0 else 0
            prob_group2 = torch.softmax(logits[group2_mask], dim=-1).mean().item() if group2_mask.sum() > 0 else 0

            prob_diffs.append(abs(prob_group1 - prob_group2))

    avg_prob_diff = sum(prob_diffs) / len(prob_diffs) if prob_diffs else 0

    avg_loss = total_loss / num_batches
    perplexity = torch.exp(torch.tensor(avg_loss))

    return avg_loss, perplexity, avg_prob_diff

### Evaluate fairness before training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

before_train_loss, before_train_ppl, before_train_fairness_diff = evaluate(model, val_dataloader, device)
print(f"\nLoRA Model - \nLoss: {before_train_loss:.4f}, \nPerplexity: {before_train_ppl:.4f}, \nFairness Difference: {before_train_fairness_diff:.6f}")

Evaluating: 100%|██████████| 8/8 [00:01<00:00,  6.08it/s]


LoRA Model - 
Loss: 2.0654, 
Perplexity: 7.8887, 
Fairness Difference: 0.000000


### Now train

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

<ipython-input-34-cf2db16cf896>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
5,25.459400
10,22.367900
15,21.002100
20,20.482300


TrainOutput(global_step=24, training_loss=21.871920267740887, metrics={'train_runtime': 8.1386, 'train_samples_per_second': 44.234, 'train_steps_per_second': 2.949, 'total_flos': 23938503505920.0, 'train_loss': 21.871920267740887, 'epoch': 3.0})

### Evaluate fairness after training

In [ ]:
after_train_loss, after_train_ppl, after_train_fairness_diff = evaluate(model, val_dataloader, device)
print(f"\nTrained LoRA Model - \nLoss: {after_train_loss:.4f}, \nPerplexity: {after_train_ppl:.4f}, \nFairness Difference: {after_train_fairness_diff:.6f}")

Evaluating: 100%|██████████| 8/8 [00:01<00:00,  6.12it/s]


Trained LoRA Model - 
Loss: 2.0698, 
Perplexity: 7.9232, 
Fairness Difference: 0.000000


## FairLoRA Implementation

There isn't a library for this or a way to add in the regularizer to the Trainer.train() so we have to go the long way around.

In [ ]:
fairlora_config = LoraConfig(
    task_type = TaskType.TOKEN_CLS,
    inference_mode = False,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.1
)

In [ ]:
fairlora_model = AutoModelForMaskedLM.from_pretrained("roberta-base")
fairlora_model = get_peft_model(fairlora_model, fairlora_config)
fairlora_model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,287,257 || trainable%: 0.4708


In [ ]:
fairtraining_args = TrainingArguments(
    output_dir="./fairlora_gender_mlm",
    eval_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    weight_decay = 0.01,
)

In [ ]:
fair_data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True
)

fair_train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size = fairtraining_args.per_device_train_batch_size,
    collate_fn = fair_data_collator,
    shuffle = True
)

fair_val_dataloader = DataLoader(
    tokenized_dataset,
    batch_size = fairtraining_args.per_device_train_batch_size,
    collate_fn = fair_data_collator,
    shuffle = False
)

In [ ]:
def fairness_loss(group1_logits, group2_logits):
    prob_group1 = torch.softmax(group1_logits, dim=-1).mean()
    prob_group2 = torch.softmax(group2_logits, dim=-1).mean()
    return torch.abs(prob_group1 - prob_group2)

optimizer = AdamW(fairlora_model.parameters(), lr=1e-5, weight_decay=0.01)
num_training_steps = len(fair_train_dataloader) * fairtraining_args.num_train_epochs
lr_scheduler = LambdaLR(optimizer, lr_lambda=lambda step: 1 - step / num_training_steps)
# lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fairlora_model.to(device)

# Fairness regularizer
lambda_fair = 0.1
fairlora_model.train()

for epoch in range(fairtraining_args.num_train_epochs):
    progress_bar = tqdm(fair_train_dataloader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        # outputs = model(**batch)
        outputs = fairlora_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        logits = outputs.logits
        mlm_loss = outputs.loss

        group1_mask = torch.tensor([
            tokenizer.decode(ids.tolist()).count("he") > 0 for ids in batch['input_ids']]).to(device)
        group1_mask = group1_mask.unsqueeze(1).expand(-1, batch['input_ids'].size(1))
        group2_mask = ~group1_mask

        group1_logits = logits[group1_mask]
        group2_logits = logits[group2_mask]

        # if group1_mask.sum() == 0 or group2_mask.sum() == 0:
        #   fairness_loss_value = torch.tensor(0.0).to(device)

        #fairness_loss_value = fairness_loss(group1_logits, group2_logits)
        if group1_logits.size(0) > 0 and group2_logits.size(0) > 0:
          fairness_loss_value = fairness_loss(group1_logits, group2_logits)
        else:
          fairness_loss_value = torch.tensor(0.0).to(device)

        total_loss = mlm_loss + lambda_fair * fairness_loss_value

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        lr_scheduler.step()

        progress_bar.set_postfix(
            mlm_loss=mlm_loss.item(),
            fairness_loss=fairness_loss_value.item(),
            total_loss=total_loss.item()
          )

Epoch 3: 100%|██████████| 15/15 [00:02<00:00,  5.85it/s, fairness_loss=0, mlm_loss=2.6, total_loss=2.6]


### Eval FairLoRA after training

In [ ]:
fair_train_loss, fair_train_ppl, fair_fairness_diff = evaluate(fairlora_model, val_dataloader, device)
print(f"FairLoRA Model - Loss: {fair_train_loss:.4f}, Perplexity: {fair_train_ppl:.4f}, Fairness Difference: {fair_fairness_diff:.6f}")

Evaluating: 100%|██████████| 8/8 [00:01<00:00,  6.46it/s]

FairLoRA Model - Loss: 2.0480, Perplexity: 7.7524, Fairness Difference: 0.000000


### Make it pretty

In [ ]:
print("\nComparison of Loss, Perplexity, and Fairness Difference:")
print("---------------------------------------------------------")
print(f"LoRA-RoBERTa (Before training)    - Loss: {before_train_loss:.4f}, Perplexity: {before_train_ppl:.4f}, Fairness Diff: {before_train_fairness_diff:.6f}")
print(f"LoRA-RoBERTa (After training)     - Loss: {after_train_loss:.4f}, Perplexity: {after_train_ppl:.4f}, Fairness Diff: {after_train_fairness_diff:.6f}")
print(f"FairLoRA Model                    - Loss: {fair_train_loss:.4f}, Perplexity: {fair_train_ppl:.4f}, Fairness Diff: {fair_fairness_diff:.6f}")
print("---------------------------------------------------------")


Comparison of Loss, Perplexity, and Fairness Difference:
---------------------------------------------------------
LoRA-RoBERTa (Before training)    - Loss: 2.0654, Perplexity: 7.8887, Fairness Diff: 0.000000
LoRA-RoBERTa (After training)     - Loss: 2.0698, Perplexity: 7.9232, Fairness Diff: 0.000000
FairLoRA Model                    - Loss: 2.0480, Perplexity: 7.7524, Fairness Diff: 0.000000
---------------------------------------------------------
